# Import libraries and directories

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable

layers = ['bronze','silver','gold']
adls = {layer: f'abfss://{layer}@carhuymdatalake.dfs.core.windows.net' for layer in layers}

date_id = dbutils.widgets.get('date_id')

adls_bronze = adls['bronze']
adls_silver = adls['silver']
adls_gold = adls['gold']

# Create Flag parameter

In [0]:
dbutils.widgets.text('incremental_flag','0')

In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')

# Creating dimensional model

### Fetch Relative Columns

In [0]:

df_source= spark.sql(f'''
    with CTE as(
        select 
            row_number() over (partition by Dealer_ID order by Date_ID desc) as row_num,
            Dealer_ID,
            DealerName
        from parquet.`{adls_silver}/car_sales/{date_id}`
    )
    select * from CTE
    where row_num = 1
''')

In [0]:
df_source.limit(10).display()

Dealer_ID,DealerName
DLR0058,Fiat do Brasil Motors
DLR0107,Land Rover Motors
DLR0129,Mia Motors
DLR0111,Lotus Motors
DLR0085,Humber Motors
DLR0001,AC Cars Motors
DLR0218,Lagonda Motors
DLR0082,Honda Motors
DLR0063,Ford do Brasil Motors
DLR0193,Tazzari Motors


### Dim_branch sink - Initial and incremental

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.dim_dealer'):
  df_sink = spark.sql(f'''
                            select * from cars_catalog.gold.dim_dealer
                            ''')
else:
    df_sink = spark.sql(f'''
                            select 1 as dim_dealer_key, Dealer_ID, DealerName 
                            from parquet.`{adls_silver}/car_sales/{date_id}`
                            where 1=0 
                            ''')

### Filtering new records and old records

In [0]:
df_filter = df_source \
.join(df_sink, df_source['Dealer_ID']== df_sink['Dealer_ID'], 'left') \
.select(
    df_source['Dealer_ID'], 
    df_source['DealerName'],
    df_sink['dim_dealer_key']
)

In [0]:
df_filter.limit(10).display()

Dealer_ID,DealerName,dim_dealer_key
DLR0058,Fiat do Brasil Motors,1
DLR0107,Land Rover Motors,2
DLR0129,Mia Motors,3
DLR0111,Lotus Motors,4
DLR0085,Humber Motors,5
DLR0001,AC Cars Motors,6
DLR0218,Lagonda Motors,7
DLR0082,Honda Motors,8
DLR0063,Ford do Brasil Motors,9
DLR0193,Tazzari Motors,10


**df_filter_old**

In [0]:
df_filter_old = df_filter.filter(col('dim_dealer_key').isNotNull())

In [0]:
df_filter_old.display()

Dealer_ID,DealerName,dim_dealer_key


**df_filter_new**

In [0]:
df_filter_new = df_filter.filter(col('dim_dealer_key').isNull()).select('Dealer_ID', df_source['DealerName'])

### Create surrogate key

In [0]:
# Fetch the max surrogate key from existing table
if (incremental_flag == '0'):
    max_dim_key = 0
else:
    max_dim_key = spark.sql('select max(dim_dealer_key) from cars_catalog.gold.dim_dealer').collect()[0][0]


**Create surrogate key column and add the max surrogate key**

In [0]:
#Spark sql row_number

# spark.sql(f'''
# select row_number() over (order by 1) as dim_key, Model_ID, model_category  
# from parquet.`{adls_silver}/car_sales`
# ''').display()

In [0]:
#Window function
# df_filter_new = df_filer_new.withColumn('dim_model_key', max_dim_model_key + row_number().over(Window.orderBy(lit(1))))

#monotonically_increasing_id
df_filter_new = df_filter_new.withColumn('dim_dealer_key',max_dim_key+1+monotonically_increasing_id())

In [0]:
df_filter_new.limit(10).display()

Dealer_ID,DealerName,dim_dealer_key
DLR0058,Fiat do Brasil Motors,1
DLR0107,Land Rover Motors,2
DLR0129,Mia Motors,3
DLR0111,Lotus Motors,4
DLR0085,Humber Motors,5
DLR0001,AC Cars Motors,6
DLR0218,Lagonda Motors,7
DLR0082,Honda Motors,8
DLR0063,Ford do Brasil Motors,9
DLR0193,Tazzari Motors,10


### Merge df_filter_old + df_filter_new

In [0]:
df_final = df_filter_old.union(df_filter_new)

In [0]:
df_final.limit(10).display()

Dealer_ID,DealerName,dim_dealer_key
DLR0058,Fiat do Brasil Motors,1
DLR0107,Land Rover Motors,2
DLR0129,Mia Motors,3
DLR0111,Lotus Motors,4
DLR0085,Humber Motors,5
DLR0001,AC Cars Motors,6
DLR0218,Lagonda Motors,7
DLR0082,Honda Motors,8
DLR0063,Ford do Brasil Motors,9
DLR0193,Tazzari Motors,10


# SCD Type 1 (Upsert)

In [0]:
if spark.catalog.tableExists("cars_catalog.gold.dim_dealer"):
    delta_tbl = DeltaTable.forPath(spark, f'{adls_gold}/dim_dealer')
    delta_tbl.alias('target').merge(df_final.alias('source'), 'target.dim_dealer_key = source.dim_dealer_key') \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll()\
        .execute()
else:
    df_final.write.format('delta')\
    .mode('overwrite')\
    .option('path',f'{adls_gold}/dim_dealer')\
    .saveAsTable('cars_catalog.gold.dim_dealer')

In [0]:
%sql
select * from cars_catalog.gold.dim_dealer limit 10 

Dealer_ID,DealerName,dim_dealer_key
DLR0058,Fiat do Brasil Motors,1
DLR0107,Land Rover Motors,2
DLR0129,Mia Motors,3
DLR0111,Lotus Motors,4
DLR0085,Humber Motors,5
DLR0001,AC Cars Motors,6
DLR0218,Lagonda Motors,7
DLR0082,Honda Motors,8
DLR0063,Ford do Brasil Motors,9
DLR0193,Tazzari Motors,10
